In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import math
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import pandas as pd
import math

import os
import torch
import torch.distributed as dist


from einops.layers.torch import Rearrange, Reduce

import torch
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

os.environ["WANDB_API_KEY"] = "KEY"
os.environ["WANDB_MODE"] = 'offline'
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import tqdm

from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader, Dataset
import random
from torch import Tensor
from torch import nn
from torch.optim import AdamW
from sklearn.preprocessing import StandardScaler
class EP_dataset(Dataset):
    def __init__(self, EP_folderpath, image_folderpath, batch_size=20):
        self.EP_folderpath = EP_folderpath
        self.image_folderpath = image_folderpath
        self.batch_size = batch_size
        self.progress_counter = 0
        self.max_pool = nn.MaxPool1d(kernel_size=5, stride=5)

        self.EP_data_list, self.image_label_list = self.EP_dataset()

    def __len__(self):
        return len(self.EP_data_list)

    def __getitem__(self, idx):
        scaler = StandardScaler()

        EP_data = self.EP_data_list[idx]
        EP_data = EP_data.T
        #print(EP_data.shape)
        label = self.image_label_list[idx]

        EP_data = scaler.fit_transform(EP_data)
        EP_data = torch.tensor(EP_data, dtype=torch.float32)
        EP_data = self.max_pool(EP_data)
        EP_data = EP_data.T

        #if random.random() > 0.3: 
        #    mask_len = random.randint(3, 10)  
        #    start = random.randint(0, 60 - mask_len)
        #    EP_data[start:start+mask_len] = 0

        #if random.random() > 0.5:
        #    fft_data = torch.fft.fft(EP_data)
        #    fft_data[10:-10] *= random.uniform(0.2, 0.8)  
        #    EP_data = torch.fft.ifft(fft_data).real
        
        #if random.random() > 0.3:
        #    mask_ch = random.randint(3, 5)
        #    channels = random.sample(range(EP_data.shape[0]), mask_ch)
        #    EP_data[channels] = 0
        
        #noise = torch.randn_like(EP_data) * 0.03 
        #EP_data += noise

        label = torch.tensor(int(label), dtype=torch.long)  

        #print(EP_data.shape)


        self.progress_counter += 1
        if self.progress_counter % 1000 == 0:
            print(f"Processed {self.progress_counter}/{len(self.EP_data_list)} items")

        return EP_data, label


    def EP_dataset(self):
        EP_segmentation_list = os.listdir(self.EP_folderpath)
        print(f'Total Dataset Number: {len(EP_segmentation_list)}')
        EP_data_list = []
        image_label_list = []

        for EP_data_dir in EP_segmentation_list:
            EP_data = np.load(os.path.join(self.EP_folderpath, EP_data_dir))

            image_label = EP_data_dir.split("_")[2]

            EP_data_list.append(EP_data)
            image_label_list.append(image_label)

        return EP_data_list, image_label_list
    
Full_dataset = EP_dataset(EP_folderpath= "/media/ubuntu/sda/data/mouse6/output/04_get_neuron_channel/segmentation_raw", image_folderpath="/media/ubuntu/sda/data/trigger/NaturalImages_new_2")
train_dataset, test_dataset = train_test_split(Full_dataset, test_size=0.2, random_state=42, shuffle=True)


image_feature = pd.read_csv("/media/ubuntu/sda/data/mouse6/output/06_visual_decoding/image_feature.csv").iloc[:, 1:]
image_feature = torch.tensor(image_feature.to_numpy(), dtype=torch.float32)

In [ ]:
class DataEmbedding(nn.Module):
    def __init__(self, c_in, d_model, dropout=0.5):
        super(DataEmbedding, self).__init__()
        
        self.value_embedding = nn.Linear(c_in, d_model)  
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, x):
        x = self.value_embedding(x)
        return self.dropout(x)
    
class DualAttentionTransformer(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.enc_embedding = DataEmbedding(configs.enc_in, configs.d_model)
        
        self.attn_layers = nn.ModuleList([
            nn.ModuleDict({
                'channel_attn': ChannelAttentionLayer(
                    d_model=configs.d_model,
                    n_heads=configs.n_heads
                ),
                'time_attn': TimeAttentionLayer(
                    d_model=configs.d_model,
                    n_heads=configs.n_heads 
                )
            }) for _ in range(configs.e_layers)
        ])
        
        self.channel_conv = nn.Sequential(
            nn.Conv1d(configs.d_model, configs.d_model*2, kernel_size=3, padding=1),
            nn.GELU(),
            nn.Conv1d(configs.d_model*2, configs.d_model, kernel_size=1)
        )

    def forward(self, x_enc):
        x = self.enc_embedding(x_enc)  
        
        for layer in self.attn_layers:
            x = layer['channel_attn'](x) 
            x = layer['time_attn'](x)    
            #x = x + self.channel_conv(x.permute(0,2,1)).permute(0,2,1)
            
        return x.permute(0, 2, 1)  

class ChannelAttentionLayer(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.5):
        super().__init__()
        self.attention = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True,
            kdim=d_model,
            vdim=d_model
        )
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model*4),
            nn.GELU(),
            nn.Linear(d_model*4, d_model),
            nn.Dropout(dropout))
        
    def forward(self, x):
        residual = x
    
        
        attn_output, _ = self.attention(
            query=x,
            key=x,
            value=x
        )
        
        x = self.norm(residual + self.dropout(attn_output))
        
        ffn_output = self.ffn(x)
        x = self.norm(x + ffn_output)
        
        return x
        
class TimeAttentionLayer(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.5):
        super().__init__()
        self.attention = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_out, _ = self.attention(x, x, x)  
        attn_out = self.dropout(attn_out)
        x = self.norm(x + attn_out)
        return x

class PatchEmbedding(nn.Module):
    def __init__(self, emb_size=20):
        super().__init__()
        self.channel_conv = nn.Identity()
        
        self.time_conv = nn.Sequential(
            nn.Conv1d(200, 64, kernel_size=25, stride=5, padding=12),  
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.AdaptiveAvgPool1d(200), 
            nn.Conv1d(64, 40, kernel_size=3, padding=1)
        )
        
        self.projection = nn.Sequential(
            nn.Linear(40, emb_size),
            nn.LayerNorm(emb_size)
        )

    def forward(self, x):
    
        x = self.channel_conv(x)  
        x = self.time_conv(x)     
        
        x = x.permute(0, 2, 1)    
        return self.projection(x)

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FlattenHead(nn.Sequential):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        return x

class Enc_eeg(nn.Sequential):
    def __init__(self, emb_size=20, **kwargs):
        super().__init__(
            PatchEmbedding(emb_size),
            FlattenHead()
        )

class Proj_eeg(nn.Sequential):
    def __init__(self, embedding_dim=4000, proj_dim=1024, drop_proj=0.5):
        super().__init__(
            nn.Linear(embedding_dim, proj_dim),
            ResidualAdd(nn.Sequential(
                nn.GELU(),
                nn.Linear(proj_dim, proj_dim),
                nn.Dropout(drop_proj),
            )),
            nn.LayerNorm(proj_dim),
        )


import logging
import torch
import torch.distributed.nn
from torch import distributed as dist, nn as nn
from torch.nn import functional as F

try:
    import horovod.torch as hvd
except ImportError:
    hvd = None


def gather_features(
    image_features,
    text_features,
    local_loss=False,
    gather_with_grad=False,
    rank=0,
    world_size=1,
    use_horovod=False,
):
    if use_horovod:
        assert hvd is not None, "Please install horovod"
        if gather_with_grad:
            all_image_features = hvd.allgather(image_features)
            all_text_features = hvd.allgather(text_features)
        else:
            with torch.no_grad():
                all_image_features = hvd.allgather(image_features)
                all_text_features = hvd.allgather(text_features)
            if not local_loss:
                # ensure grads for local rank when all_* features don't have a gradient
                gathered_image_features = list(
                    all_image_features.chunk(world_size, dim=0)
                )
                gathered_text_features = list(
                    all_text_features.chunk(world_size, dim=0)
                )
                gathered_image_features[rank] = image_features
                gathered_text_features[rank] = text_features
                all_image_features = torch.cat(gathered_image_features, dim=0)
                all_text_features = torch.cat(gathered_text_features, dim=0)
    else:
        # We gather tensors from all gpus
        if gather_with_grad:
            all_image_features = torch.cat(
                torch.distributed.nn.all_gather(image_features), dim=0
            )
            all_text_features = torch.cat(
                torch.distributed.nn.all_gather(text_features), dim=0
            )
        else:
            gathered_image_features = [
                torch.zeros_like(image_features) for _ in range(world_size)
            ]
            gathered_text_features = [
                torch.zeros_like(text_features) for _ in range(world_size)
            ]
            dist.all_gather(gathered_image_features, image_features)
            dist.all_gather(gathered_text_features, text_features)
            if not local_loss:
                # ensure grads for local rank when all_* features don't have a gradient
                gathered_image_features[rank] = image_features
                gathered_text_features[rank] = text_features
            all_image_features = torch.cat(gathered_image_features, dim=0)
            all_text_features = torch.cat(gathered_text_features, dim=0)

    return all_image_features, all_text_features

class ClipLoss(nn.Module):
    def __init__(
        self,
        local_loss=False,
        gather_with_grad=False,
        cache_labels=False,
        rank=0,
        world_size=1,
        use_horovod=False,
    ):
        super().__init__()
        self.local_loss = local_loss
        self.gather_with_grad = gather_with_grad
        self.cache_labels = cache_labels
        self.rank = rank
        self.world_size = world_size
        self.use_horovod = use_horovod

        # cache state
        self.prev_num_logits = 0
        self.labels = {}

    def forward(self, image_features, text_features, logit_scale):
        device = image_features.device
        if self.world_size > 1:
            all_image_features, all_text_features = gather_features(
                image_features,
                text_features,
                self.local_loss,
                self.gather_with_grad,
                self.rank,
                self.world_size,
                self.use_horovod,
            )

            if self.local_loss:
                logits_per_image = logit_scale * image_features @ all_text_features.T
                logits_per_text = logit_scale * text_features @ all_image_features.T
            else:
                logits_per_image = (
                    logit_scale * all_image_features @ all_text_features.T
                )
                logits_per_text = logits_per_image.T
        else:
            logits_per_image = logit_scale * image_features @ text_features.T
            logits_per_text = logit_scale * text_features @ image_features.T

        num_logits = logits_per_image.shape[0]
        if self.prev_num_logits != num_logits or device not in self.labels:
            labels = torch.arange(num_logits, device=device, dtype=torch.long)
            if self.world_size > 1 and self.local_loss:
                labels = labels + num_logits * self.rank
            if self.cache_labels:
                self.labels[device] = labels
                self.prev_num_logits = num_logits
        else:
            labels = self.labels[device]

        total_loss = (
            F.cross_entropy(logits_per_image, labels)
            + F.cross_entropy(logits_per_text, labels)
        ) / 2
        return total_loss

class global_config:
    def __init__(self):
        self.task_name = 'classification'
        self.enc_in = 30       
        self.d_model = 200     
        self.dropout = 0.4   
        self.n_heads = 2
        self.e_layers = 2   


class EP_encoder(nn.Module):    
    def __init__(self, global_config):
        super().__init__()
        self.encoder = DualAttentionTransformer(global_config)   
        self.enc_eeg = Enc_eeg()
        self.proj_eeg = Proj_eeg(embedding_dim=20*50*4)  
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1/0.07))

        self.loss_func = ClipLoss()

    def forward(self, x):
        x = self.encoder(x)       
        eeg_embedding = self.enc_eeg(x) 
        return self.proj_eeg(eeg_embedding)

device = 'cuda'
config = global_config() 
EP_encoder_v1 = EP_encoder(config).to(device)

batch_size_per_gpu = 24
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size_per_gpu,
    shuffle=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size_per_gpu,
    shuffle=False
)

def train_model(encoder_modle, dataloader, optimizer, device, image_feature):
    encoder_modle.train()
    total_loss = 0
    correct = 0
    total = 0
    EP_feature_list = []
    predict_label = []
    actual_label = []
    correct_top5 = 0
    image_feature = image_feature.to(device)
    img_features_all = image_feature.to(device).float()
    for batch_id, (EP_data, label) in enumerate(dataloader):

        EP_data = EP_data.to(device)
        label = label.to(device)
        batch_size = EP_data.size(0)
        selected_image_features = image_feature[label - 1]
        #print(selected_image_features.shape)

        optimizer.zero_grad()
        EP_feature = encoder_modle(EP_data)
        EP_feature_list.append(EP_feature)
        logit_scale = encoder_modle.logit_scale

        EP_feature = F.normalize(EP_feature, dim=-1)
        selected_image_features = F.normalize(selected_image_features, dim=-1)
        
        img_loss = encoder_modle.loss_func(selected_image_features,EP_feature, encoder_modle.logit_scale)
        loss = img_loss
        loss.backward()
        
        img_features_all = F.normalize(img_features_all, dim = -1)
        logits_img = logit_scale * EP_feature @ img_features_all.T
        logits_single = logits_img
        predicted = torch.argmax(logits_single, dim=1)
        predicted += 1


        _, top5_predicted = torch.topk(logits_single, 5, dim=1)
        top5_predicted += 1
        for i in range(batch_size):
            if label[i] in top5_predicted[i]:
                correct_top5 += 1

        optimizer.step()
        #print("train_loss:")
        #print(img_loss.item())
        total_loss += img_loss.item()
        total += batch_size

        predict_label.append(predicted)
        actual_label.append(label)

    predict_label = torch.cat(predict_label, dim=0)
    actual_label = torch.cat(actual_label, dim=0)

    correct += (predict_label == actual_label).sum().item()

    average_loss = total_loss / (batch_id+1)
    accuracy = correct / total
    accuracy_top5 = correct_top5/total

    return average_loss, accuracy, torch.cat(EP_feature_list, dim=0), accuracy_top5
    
def evaluate_model(encoder_modle, dataloader, device, image_feature):
    encoder_modle.eval()

    total_loss = 0
    correct = 0
    total = 0
    EP_feature_list = []
    predict_label = []
    actual_label = []
    image_feature = image_feature.to(device)
    img_features_all = image_feature.to(device).float()
    correct_top5 = 0
    
    with torch.no_grad():
        for batch_id, (EP_data, label) in enumerate(dataloader):

            EP_data = EP_data.to(device)
            label = label.to(device)
            selected_image_features = image_feature[label - 1]
            batch_size = EP_data.size(0)
            logit_scale = encoder_modle.logit_scale

            EP_feature = encoder_modle(EP_data)

            EP_feature_list.append(EP_feature)

            EP_feature = F.normalize(EP_feature, dim=-1)
            selected_image_features = F.normalize(selected_image_features, dim=-1)
            
            img_loss = encoder_modle.loss_func(selected_image_features,EP_feature, encoder_modle.logit_scale)
            
            total_loss += img_loss.item()

            img_features_all = F.normalize(img_features_all, dim = -1)
            logits_img = logit_scale * EP_feature @ img_features_all.T
            logits_single = logits_img
            predicted = torch.argmax(logits_single, dim=1)
            predicted += 1

            _, top5_predicted = torch.topk(logits_single, 5, dim=1)
            top5_predicted += 1
            for i in range(batch_size):
                if label[i] in top5_predicted[i]:
                    correct_top5 += 1
                    
            total += batch_size
    
            predict_label.append(predicted)
            actual_label.append(label)
    
        predict_label = torch.cat(predict_label, dim=0)
        actual_label = torch.cat(actual_label, dim=0)
    
        correct += (predict_label == actual_label).sum().item()
    
        average_loss = total_loss / (batch_id+1)
        accuracy = correct / total
        accuracy_top5 = correct_top5/total

        return average_loss, accuracy, predict_label, actual_label, accuracy_top5
        
import torch.optim as optim

optimizer = optim.AdamW([
    {'params': EP_encoder_v1.encoder.parameters(), 'lr': 1e-3},
    {'params': EP_encoder_v1.proj_eeg.parameters(), 'lr': 5e-3}
], weight_decay=0.001)  

scheduler = optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers=[
        optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=10),
        optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    ],
    milestones=[10]
)

def main_train_loop(date, encoder_modle, train_loader, test_loader, optimizer, device, config, image_feature):
    print('start train')
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    results = []

    for epoch in range(30):
        print(epoch + 1)
        train_loss, train_accuracy, features_tensor, train_accuracy_top5 = train_model(encoder_modle, train_loader, optimizer, device, image_feature)
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)

        print('end_train')
        test_loss, test_accuracy, test_predict, test_actual, accuracy_top5 = evaluate_model(encoder_modle, test_loader, device, image_feature)
        
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        scheduler.step()

        if (epoch +1) % 5 == 0:     
            print(f'Epoch: {epoch + 1}')
            os.makedirs(f"./models/'classification'/{date}", exist_ok=True)             
            file_path = f"./models/'classification'/{date}/{epoch+1}.pth"
            torch.save(encoder_modle.state_dict(), file_path)  


            print(f"model saved in {file_path}!")
        
        print(f"train loss: {train_loss} at epoch: {epoch + 1}")
        print(f"train accuracy: {train_accuracy} at epoch: {epoch + 1}")
        print(f"train top5-accuracy: {train_accuracy_top5} at epoch: {epoch + 1}")
        print(f"test loss: {test_loss} at epoch: {epoch + 1}")
        print(f"test accuracy: {test_accuracy} at epoch: {epoch + 1}")
        print(f"test top5-accuracy: {accuracy_top5} at epoch: {epoch + 1}")
        print("----------")
            
        epoch_results = {
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_accuracy,
        "test_loss": test_loss,
        "test_accuracy": test_accuracy,
        "test_predict": test_predict,
        "test_actual": test_actual
        }
    
    results.append(epoch_results)

    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Loss curve
    axs[0].plot(train_losses, label='Train Loss')
    axs[0].plot(test_losses, label='Test Loss')
    axs[0].legend()
    axs[0].set_title("Loss Curve")

    # Overall accuracy curve
    axs[1].plot(train_accuracies, label='Train Accuracy')
    axs[1].plot(test_accuracies, label='Test Accuracy')
    axs[1].legend()
    axs[1].set_title("Accuracy Curve")

    plt.savefig(f'{date}_curve.png')

    return results

results = main_train_loop(encoder_modle=EP_encoder_v1, train_loader=train_loader, test_loader=test_loader, optimizer=optimizer, device=device, config=global_config,date='20250309_raw', image_feature=image_feature)

import pickle
with open('results_20250309_raw.pkl', 'wb') as f:
    pickle.dump(results, f)